# Installations

In [ ]:
!pip install torch torchvision

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

# Download shape_predictor_68_face_landmarks.dat file from dlib

In [ ]:
!wget http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
!bzip2 -d shape_predictor_68_face_landmarks.dat.bz2

--2024-06-30 10:05:42--  http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
Resolving dlib.net (dlib.net)... 107.180.26.78
Connecting to dlib.net (dlib.net)|107.180.26.78|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64040097 (61M)
Saving to: ‘shape_predictor_68_face_landmarks.dat.bz2’

shape_predictor_68_ 100%[===================>]  61.07M  40.1MB/s    in 1.5s    

2024-06-30 10:05:43 (40.1 MB/s) - ‘shape_predictor_68_face_landmarks.dat.bz2’ saved [64040097/64040097]



# Imports

In [ ]:
import torch
from torchvision import datasets, transforms, utils
import dlib
import numpy as np
import os
import cv2
from google.colab.patches import cv2_imshow
from PIL import Image
import matplotlib.pyplot as plt

# Definitions

In [ ]:
# Create a directory to store the landmarks
landmarks_dir = './landmarks'
os.makedirs(landmarks_dir, exist_ok=True)

In [ ]:
# Initialize dlib's face detector and facial landmark predictor
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

# Define a transform to convert the images to tensors
transform = transforms.Compose([
    transforms.ToTensor()
])

In [ ]:
def getLocations(chin, facial_feature):
  facial_feature_center = (facial_feature[0] + facial_feature[1]) / 2
  feature_cl = facial_feature_center - chin[0]
  feature_cr = chin[2] - facial_feature_center
  feature_cb = chin[1] - facial_feature_center
  print("cl, cr, cb:", feature_cl[0], feature_cr[0], feature_cb[1])
  return feature_cl[0], feature_cr[0], feature_cb[1]

In [ ]:
# Function to detect facial landmarks and save them
def detect_landmarks(image, name, idx):
    # Convert tensor image to PIL image
    pil_image = transforms.ToPILImage()(image)

    # Convert PIL image to numpy array
    img_np = np.array(pil_image)

    # Get image dimensions
    img_height, img_width = img_np.shape[:2]

    # Convert image to grayscale
    gray = cv2.cvtColor(img_np, cv2.COLOR_RGB2GRAY)

    # Detect faces in the grayscale image
    faces = detector(gray)

    # If no face detected, return None
    if len(faces) == 0:
        return None

    # Only process the first detected face
    face = faces[0]

    # Predict facial landmarks
    landmarks = predictor(gray, face)

    chin_indices = [0, 8, 16] # left edge, bottom edge and right edge of the chin
    # (left and rigth ends):
    left_eye_indices = [36, 39]
    right_eye_indices = [42, 45]
    nose_indices = [31, 35]
    mouth_indices = [48, 54]

    # Extract landmarks as numpy arrays
    chin_np = np.array([(landmarks.part(i).x, landmarks.part(i).y) for i in chin_indices])
    left_eye_np = np.array([(landmarks.part(i).x, landmarks.part(i).y) for i in left_eye_indices])
    right_eye_np = np.array([(landmarks.part(i).x, landmarks.part(i).y) for i in right_eye_indices])
    nose_np = np.array([(landmarks.part(i).x, landmarks.part(i).y) for i in nose_indices])
    mouth_np = np.array([(landmarks.part(i).x, landmarks.part(i).y) for i in mouth_indices])

    # Normalize landmarks
    chin_normalized = chin_np / np.array([img_width, img_height])
    left_eye_normalized = left_eye_np / np.array([img_width, img_height])
    right_eye_normalized = right_eye_np / np.array([img_width, img_height])
    nose_normalized = nose_np / np.array([img_width, img_height])
    mouth_normalized = mouth_np / np.array([img_width, img_height])

    # Create a valid filename by combining the name and index
    valid_filename = f"{name}_{idx}.txt".replace(' ', '_')

    el_cl, el_cr, el_cb = getLocations(chin_normalized, left_eye_normalized)
    er_cl, er_cr, er_cb = getLocations(chin_normalized, right_eye_normalized)
    n_cl, n_cr, n_cb = getLocations(chin_normalized, nose_normalized)
    m_cl, m_cr, m_cb = getLocations(chin_normalized, mouth_normalized)

    el = np.array([el_cl, el_cr, el_cb])
    er = np.array([er_cl, er_cr, er_cb])
    n = np.array([n_cl, n_cr, n_cb])
    m = np.array([m_cl, m_cr, m_cb])

    results = np.array([el, er, n, m])

    # Format the results into a string
    output = ''
    for arr in results:
        output += ' '.join(map(str, arr)) + '\n'

    # Write the formatted string to a text file
    with open(f'{landmarks_dir}/{valid_filename}', 'w') as f:
        f.write(output)

# Detect facial landmarks (Custom Dataset)

In [ ]:
# Klasördeki tüm dosyaları alalım
files = os.listdir("/content/frontal_faces")

# Görüntü dosyalarını işle
for idx, file_name in enumerate(files):
    # Dosya yolunu oluştur
    file_path = os.path.join("/content/frontal_faces", file_name)

    # Görüntüyü yükle
    image = cv2.imread(file_path)

    # Görüntüyü RGB formatına dönüştür
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Görüntüyü tensor'a dönüştür
    image_tensor = transforms.ToTensor()(image_rgb)

    # Landmarkları tespit et ve kaydet
    detect_landmarks(image_tensor, file_name.split('.')[0], idx)
print("Landmarks extraction completed!")

cl, cr, cb: 0.1403846153846154 0.4673076923076923 0.4769230769230769
cl, cr, cb: 0.4076923076923077 0.20000000000000007 0.5019230769230769
cl, cr, cb: 0.2730769230769231 0.33461538461538465 0.32499999999999996
cl, cr, cb: 0.2846153846153846 0.3230769230769231 0.1942307692307692
Landmarks extraction completed!


# Save the results

In [ ]:
# Download landmark annotations
!zip -r landmarks.zip /content/landmarks